In [42]:
!pip install streamlit
!pip install pyvi
!pip install datasets

In [55]:
%%writefile demo.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from pyvi import ViTokenizer
import torch
from datasets import load_dataset

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
model = AutoModelForSeq2SeqLM.from_pretrained("MinhViet/Bartpho").to(device)
data = load_dataset("Valleyy/final_nlp_data")

def get_label_summary(input):
    for i in range(len(data['valid'])):
        if data['valid']['Document'][i] == input:
            return data['valid']['Summary'][i]
    return "Not Found"

st.markdown("""
    <style>
    .center-text {
        text-align: center;
    }
    </style>
    """, unsafe_allow_html=True)

st.markdown("<h1 class='center-text'>Abstractive text summarization</h1>", unsafe_allow_html=True)

if 'submitted_question' not in st.session_state:
    st.session_state.submitted_question = ''

question = st.text_area("Input text", st.session_state.submitted_question, height=300)
segment_question = ViTokenizer.tokenize(question)

confirm_button = st.button("Confirm")
clear_button = st.button("Clear")

if confirm_button and question:
    st.session_state.submitted_question = segment_question

if clear_button:
    st.session_state.submitted_question = ''
    question = ''  # Clear the input text area

if st.session_state.submitted_question:
    # Generate summary using the model
    encoded_input = tokenizer(st.session_state.submitted_question, return_tensors="pt").to(device)
    output = model.generate(input_ids=encoded_input['input_ids'], attention_mask=encoded_input['attention_mask'], max_length=512, num_beams=5)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    clean_response = response.replace('_', ' ')

    # Retrieve the label summary
    label_summary = get_label_summary(question)

    st.text_area("Generated summary", clean_response, height=100)
    st.text_area("Label summary", label_summary, height=100)


Overwriting demo.py


In [56]:
! wget -q -O - ipv4.icanhazip.com

35.197.119.74


In [57]:
! streamlit run demo.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.197.119.74:8501

npx: installed 22 in 1.647s
your url is: https://salty-jokes-jump.loca.lt
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  Stopping...
^C
